# 模型知識編輯 (Model Editing)

本 notebook 對應李宏毅老師 2025 Spring ML HW8，探討如何精準修改 LLM 中儲存的知識。

## 學習目標

1. 理解知識在 LLM 中的儲存方式
2. 學習 ROME（Rank-One Model Editing）方法
3. 了解 MEMIT（Mass-Editing Memory In a Transformer）
4. 實作簡單的知識編輯
5. 評估編輯效果與副作用

## 參考資源

- [ROME Paper](https://arxiv.org/abs/2202.05262) - Locating and Editing Factual Associations in GPT
- [MEMIT Paper](https://arxiv.org/abs/2210.07229) - Mass-Editing Memory in a Transformer
- [EasyEdit](https://github.com/zjunlp/EasyEdit) - 知識編輯工具庫
- [2025 Spring HW8](https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php)

## 1. 為什麼需要模型編輯？

### 1.1 LLM 知識更新的挑戰

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    LLM 知識更新的問題                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  問題場景：                                                              │
│  ─────────                                                              │
│  • LLM 訓練資料有截止日期（knowledge cutoff）                             │
│  • 世界知識會隨時間改變                                                   │
│  • 模型可能學到錯誤資訊                                                   │
│  • 需要移除有害或敏感資訊                                                 │
│                                                                         │
│  範例：                                                                  │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  Q: Who is the Prime Minister of UK?                        │       │
│  │                                                             │       │
│  │  舊模型: Boris Johnson (2019-2022)                          │       │
│  │  需更新: Keir Starmer (2024-)                               │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
│  傳統方法的問題：                                                        │
│  ─────────────────                                                      │
│  1. 重新訓練：成本極高（數百萬美元）                                       │
│  2. Fine-tuning：可能導致災難性遺忘                                       │
│  3. RAG：只能補充，無法修正模型內部知識                                    │
│                                                                         │
│  模型編輯的目標：                                                        │
│  ───────────────                                                        │
│  精準修改特定知識，同時保持其他能力不變                                    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 1.2 編輯目標的三個維度

| 維度 | 定義 | 範例 |
|------|------|------|
| **Efficacy** | 編輯後正確回答目標問題 | Q: UK PM? → A: Keir Starmer |
| **Generalization** | 能泛化到相關問題 | Q: Who leads UK? → A: Keir Starmer |
| **Specificity** | 不影響不相關知識 | Q: France PM? → A: (保持不變) |

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from copy import deepcopy

# 設定
torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. 知識在 Transformer 中的儲存

### 2.1 知識定位 (Knowledge Localization)

研究發現，事實知識主要儲存在 Transformer 的 **MLP 層**中。

```
┌─────────────────────────────────────────────────────────────────────────┐
│                   Transformer Layer 中的知識儲存                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Transformer Block:                                                     │
│  ┌─────────────────────────────────────────────────────────────────┐   │
│  │                                                                 │   │
│  │   Input x                                                       │   │
│  │      │                                                          │   │
│  │      ▼                                                          │   │
│  │  ┌─────────────┐                                                │   │
│  │  │  Attention  │ ← 主要處理 token 間的關係                        │   │
│  │  │   (QKV)     │                                                │   │
│  │  └──────┬──────┘                                                │   │
│  │         │ + x (residual)                                        │   │
│  │         ▼                                                       │   │
│  │  ┌─────────────┐                                                │   │
│  │  │    MLP      │ ← 主要儲存事實知識 ⭐                            │   │
│  │  │  W_up, W_down                                                │   │
│  │  └──────┬──────┘                                                │   │
│  │         │ + (residual)                                          │   │
│  │         ▼                                                       │   │
│  │      Output                                                     │   │
│  │                                                                 │   │
│  └─────────────────────────────────────────────────────────────────┘   │
│                                                                         │
│  MLP 可以看作 Key-Value Memory:                                         │
│  ──────────────────────────────                                         │
│  MLP(x) = W_down · σ(W_up · x)                                         │
│                                                                         │
│  其中：                                                                  │
│  • W_up 的列向量 = Keys（輸入模式）                                       │
│  • W_down 的行向量 = Values（輸出表示）                                   │
│  • σ = 激活函數                                                         │
│                                                                         │
│  知識編輯 = 修改特定的 Key-Value 對應關係                                 │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
class SimpleMLP(nn.Module):
    """
    簡化的 MLP 層，展示知識儲存概念
    """
    def __init__(self, d_model: int, d_hidden: int):
        super().__init__()
        self.d_model = d_model
        self.d_hidden = d_hidden
        
        # Up projection: d_model -> d_hidden
        self.W_up = nn.Linear(d_model, d_hidden, bias=False)
        
        # Down projection: d_hidden -> d_model
        self.W_down = nn.Linear(d_hidden, d_model, bias=False)
        
        # 激活函數
        self.activation = nn.GELU()
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [batch, seq, d_model]
        """
        hidden = self.activation(self.W_up(x))  # [batch, seq, d_hidden]
        output = self.W_down(hidden)            # [batch, seq, d_model]
        return output
    
    def get_key_value_interpretation(self):
        """
        將 MLP 解釋為 Key-Value memory
        """
        # Keys: W_up 的列向量 (d_hidden 個 keys，每個維度 d_model)
        keys = self.W_up.weight.data  # [d_hidden, d_model]
        
        # Values: W_down 的行向量 (d_hidden 個 values，每個維度 d_model)
        values = self.W_down.weight.data.T  # [d_hidden, d_model]
        
        return keys, values


# 示範
d_model = 128
d_hidden = 512

mlp = SimpleMLP(d_model, d_hidden)
keys, values = mlp.get_key_value_interpretation()

print(f"MLP 維度: d_model={d_model}, d_hidden={d_hidden}")
print(f"Keys shape: {keys.shape} (每個 key 是一個 {d_model} 維向量)")
print(f"Values shape: {values.shape} (每個 value 是一個 {d_model} 維向量)")
print(f"\n可以把 MLP 看作 {d_hidden} 個 Key-Value pairs 的記憶體")

## 3. ROME: Rank-One Model Editing

### 3.1 ROME 核心思想

ROME 透過 **rank-one update** 來修改 MLP 的權重，使其輸出新的知識。

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         ROME 方法概覽                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  目標：將 (subject, relation) → old_object 改為 → new_object            │
│  例如：(UK Prime Minister, is) → Boris Johnson → Keir Starmer          │
│                                                                         │
│  步驟：                                                                  │
│  ──────                                                                 │
│                                                                         │
│  Step 1: 找到關鍵層 (Causal Tracing)                                    │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  透過因果介入實驗，找出哪一層對特定事實最重要                     │       │
│  │  通常是中間層 (e.g., layer 15-20 in GPT-2 XL)                 │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
│  Step 2: 計算 key vector k*                                            │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  k* = average hidden state at subject's last token          │       │
│  │       across different prompts containing the subject       │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
│  Step 3: 計算 target value v*                                          │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  找到使模型輸出新答案所需的 hidden state                       │       │
│  │  v* = argmin ||h - v|| s.t. model outputs new_object       │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
│  Step 4: Rank-One Update                                               │
│  ┌─────────────────────────────────────────────────────────────┐       │
│  │  W_new = W_old + Δ                                          │       │
│  │  Δ = (v* - W_old·k*) · k*ᵀ · (C + k*·k*ᵀ)⁻¹               │       │
│  │                                                             │       │
│  │  其中 C 是統計矩陣，確保更新最小化對其他 key 的影響           │       │
│  └─────────────────────────────────────────────────────────────┘       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
@dataclass
class EditRequest:
    """知識編輯請求"""
    subject: str          # 主詞 (e.g., "UK Prime Minister")
    relation: str         # 關係 (e.g., "is")
    old_object: str       # 舊答案 (e.g., "Boris Johnson")
    new_object: str       # 新答案 (e.g., "Keir Starmer")
    prompts: List[str]    # 用於生成 key 的 prompts


def compute_rank_one_update(
    W: torch.Tensor,        # 原始權重 [out_dim, in_dim]
    k_star: torch.Tensor,   # 目標 key [in_dim]
    v_star: torch.Tensor,   # 目標 value [out_dim]
    C: torch.Tensor = None, # 統計矩陣 [in_dim, in_dim]
    lambda_reg: float = 1e-4
) -> torch.Tensor:
    """
    計算 ROME 的 rank-one update
    
    W_new = W + (v* - W·k*) · k*ᵀ · (C + k*·k*ᵀ)⁻¹
    
    這個更新確保：
    - W_new · k* ≈ v* (新知識)
    - W_new · k ≈ W · k for k ≠ k* (保持其他知識)
    """
    in_dim = W.shape[1]
    
    # 如果沒有提供 C，使用單位矩陣
    if C is None:
        C = torch.eye(in_dim, device=W.device) * lambda_reg
    
    # 確保維度正確
    k_star = k_star.view(-1)  # [in_dim]
    v_star = v_star.view(-1)  # [out_dim]
    
    # 計算殘差：v* - W·k*
    residual = v_star - W @ k_star  # [out_dim]
    
    # 計算 (C + k*·k*ᵀ)⁻¹ · k*
    k_outer = torch.outer(k_star, k_star)  # [in_dim, in_dim]
    inv_term = torch.linalg.solve(C + k_outer, k_star)  # [in_dim]
    
    # Rank-one update: residual · inv_termᵀ
    delta = torch.outer(residual, inv_term)  # [out_dim, in_dim]
    
    return delta


# 示範 rank-one update
print("Rank-One Update 示範：")
print("="*50)

# 創建一個簡單的線性層
in_dim, out_dim = 10, 8
W = torch.randn(out_dim, in_dim)

# 目標：讓特定輸入 k_star 產生特定輸出 v_star
k_star = torch.randn(in_dim)
v_star = torch.randn(out_dim)

print(f"原始輸出 W @ k_star: {(W @ k_star)[:5]}...")
print(f"目標輸出 v_star: {v_star[:5]}...")

# 計算更新
delta = compute_rank_one_update(W, k_star, v_star)
W_new = W + delta

print(f"\n更新後輸出 W_new @ k_star: {(W_new @ k_star)[:5]}...")
print(f"誤差: {torch.norm(W_new @ k_star - v_star).item():.6f}")

# 檢查對其他 key 的影響
k_other = torch.randn(in_dim)
print(f"\n對其他 key 的影響:")
print(f"原始: {(W @ k_other)[:5]}...")
print(f"更新後: {(W_new @ k_other)[:5]}...")
print(f"差異: {torch.norm(W_new @ k_other - W @ k_other).item():.6f}")

## 4. 簡化版知識編輯實作

### 4.1 建立簡單的知識模型

In [ ]:
class SimpleKnowledgeModel(nn.Module):
    """
    簡化的知識模型，用於展示編輯概念
    
    模型學習 (subject_embedding) → (object_embedding) 的映射
    """
    def __init__(self, 
                 vocab_size: int = 100,
                 embed_dim: int = 64,
                 hidden_dim: int = 128):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Knowledge MLP (這是我們要編輯的層)
        self.knowledge_mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # Output projection
        self.output = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, subject_ids: torch.Tensor) -> torch.Tensor:
        """
        subject_ids: [batch_size]
        Returns: logits [batch_size, vocab_size]
        """
        # Embed subject
        x = self.embedding(subject_ids)  # [batch, embed_dim]
        
        # Apply knowledge MLP
        x = self.knowledge_mlp(x)  # [batch, embed_dim]
        
        # Output logits
        logits = self.output(x)  # [batch, vocab_size]
        
        return logits
    
    def predict(self, subject_ids: torch.Tensor) -> torch.Tensor:
        """預測 object id"""
        logits = self.forward(subject_ids)
        return logits.argmax(dim=-1)
    
    def get_hidden(self, subject_ids: torch.Tensor) -> torch.Tensor:
        """取得 MLP 輸入的 hidden state"""
        return self.embedding(subject_ids)


# 建立模型
vocab_size = 100
model = SimpleKnowledgeModel(vocab_size=vocab_size).to(device)
print(f"模型參數量: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# 建立知識資料集
knowledge_facts = {
    # subject_id: object_id
    0: 50,   # "UK_PM" -> "Boris_Johnson"
    1: 51,   # "France_PM" -> "Macron"
    2: 52,   # "Germany_Chancellor" -> "Scholz"
    3: 53,   # "US_President" -> "Biden"
    4: 54,   # "Japan_PM" -> "Kishida"
}

# 訓練模型學習這些知識
def train_knowledge_model(model, facts, epochs=500, lr=0.01):
    """訓練模型記住知識"""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    subjects = torch.tensor(list(facts.keys())).to(device)
    objects = torch.tensor(list(facts.values())).to(device)
    
    losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        logits = model(subjects)
        loss = criterion(logits, objects)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        if (epoch + 1) % 100 == 0:
            acc = (model.predict(subjects) == objects).float().mean()
            print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Accuracy = {acc.item():.2%}")
    
    return losses


print("訓練模型記住知識...")
losses = train_knowledge_model(model, knowledge_facts)

# 驗證
print("\n驗證知識：")
for subject, expected_object in knowledge_facts.items():
    predicted = model.predict(torch.tensor([subject]).to(device)).item()
    print(f"Subject {subject} -> Predicted: {predicted}, Expected: {expected_object}, "
          f"{'✓' if predicted == expected_object else '✗'}")

In [ ]:
class KnowledgeEditor:
    """
    簡化版知識編輯器
    """
    def __init__(self, model: SimpleKnowledgeModel):
        self.model = model
        self.original_state = deepcopy(model.state_dict())
    
    def edit(self, 
             subject_id: int, 
             new_object_id: int,
             method: str = "fine_tune") -> Dict:
        """
        編輯知識
        
        Args:
            subject_id: 要編輯的 subject
            new_object_id: 新的 object
            method: 編輯方法 ('fine_tune' 或 'rank_one')
        """
        if method == "fine_tune":
            return self._edit_finetune(subject_id, new_object_id)
        elif method == "rank_one":
            return self._edit_rank_one(subject_id, new_object_id)
        else:
            raise ValueError(f"Unknown method: {method}")
    
    def _edit_finetune(self, subject_id: int, new_object_id: int, 
                       steps: int = 100, lr: float = 0.01) -> Dict:
        """
        使用 fine-tuning 編輯（基線方法）
        問題：可能導致災難性遺忘
        """
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        
        subject = torch.tensor([subject_id]).to(device)
        target = torch.tensor([new_object_id]).to(device)
        
        for _ in range(steps):
            optimizer.zero_grad()
            logits = self.model(subject)
            loss = criterion(logits, target)
            loss.backward()
            optimizer.step()
        
        return {"method": "fine_tune", "steps": steps}
    
    def _edit_rank_one(self, subject_id: int, new_object_id: int) -> Dict:
        """
        使用 rank-one update 編輯（類似 ROME）
        """
        with torch.no_grad():
            # 取得 subject 的 hidden representation (作為 key)
            subject = torch.tensor([subject_id]).to(device)
            k_star = self.model.embedding(subject).squeeze()  # [embed_dim]
            
            # 取得目標 object 的 embedding (作為 target output)
            # 注意：這是簡化版，實際 ROME 會用更複雜的方法計算 v*
            target_embedding = self.model.output.weight[new_object_id]  # [embed_dim]
            v_star = target_embedding
            
            # 編輯 knowledge_mlp 的最後一層
            # 取得最後一個 Linear 層
            last_linear = self.model.knowledge_mlp[-1]
            W = last_linear.weight.data  # [embed_dim, hidden_dim]
            
            # 取得進入最後一層的 hidden state
            x = self.model.embedding(subject)
            for layer in list(self.model.knowledge_mlp)[:-1]:
                x = layer(x)
            h = x.squeeze()  # [hidden_dim] - 這是實際的 key
            
            # 計算 rank-one update
            delta = compute_rank_one_update(W, h, v_star)
            
            # 應用更新
            last_linear.weight.data = W + delta
        
        return {"method": "rank_one"}
    
    def restore(self):
        """恢復原始模型"""
        self.model.load_state_dict(deepcopy(self.original_state))


# 測試知識編輯
print("\n" + "="*60)
print("知識編輯測試")
print("="*60)

editor = KnowledgeEditor(model)

# 編輯前
print("\n編輯前預測：")
for subject, expected in knowledge_facts.items():
    pred = model.predict(torch.tensor([subject]).to(device)).item()
    print(f"Subject {subject}: {pred}")

# 執行編輯：將 subject 0 的答案從 50 改為 60
print("\n執行編輯：Subject 0: 50 -> 60")
edit_result = editor.edit(subject_id=0, new_object_id=60, method="rank_one")
print(f"編輯方法: {edit_result['method']}")

# 編輯後
print("\n編輯後預測：")
for subject, expected in knowledge_facts.items():
    pred = model.predict(torch.tensor([subject]).to(device)).item()
    if subject == 0:
        status = "✓ (已編輯)" if pred == 60 else "✗"
    else:
        status = "✓ (保持)" if pred == expected else "✗ (受影響)"
    print(f"Subject {subject}: {pred} {status}")

## 5. MEMIT: 批量知識編輯

### 5.1 MEMIT 概念

MEMIT 擴展了 ROME，可以同時編輯多條知識。

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         MEMIT vs ROME                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ROME:                                                                  │
│  • 一次編輯一條知識                                                      │
│  • 修改單一層                                                           │
│  • 適合小規模編輯                                                        │
│                                                                         │
│  MEMIT:                                                                 │
│  • 一次編輯多條知識                                                      │
│  • 將編輯分散到多層                                                      │
│  • 更好的 scalability                                                   │
│                                                                         │
│  MEMIT 更新公式：                                                        │
│  ───────────────                                                        │
│  對於 N 條編輯，在 L 層分散更新：                                          │
│                                                                         │
│  W_new^l = W_old^l + R · K^T · (C + K·K^T)^{-1}                        │
│                                                                         │
│  其中：                                                                  │
│  • K = [k_1, k_2, ..., k_N] 是所有編輯的 key 向量                        │
│  • R = [r_1, r_2, ..., r_N] 是對應的殘差向量                             │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
def batch_rank_update(
    W: torch.Tensor,          # [out_dim, in_dim]
    K: torch.Tensor,          # [num_edits, in_dim] - 多個 keys
    V: torch.Tensor,          # [num_edits, out_dim] - 多個 target values
    lambda_reg: float = 1e-4
) -> torch.Tensor:
    """
    批量 rank 更新（簡化版 MEMIT）
    
    W_new = W + (V - W·K^T)^T · K · (λI + K^T·K)^{-1}
    """
    num_edits, in_dim = K.shape
    out_dim = W.shape[0]
    
    # 計算當前輸出
    current_output = W @ K.T  # [out_dim, num_edits]
    
    # 計算殘差
    residual = V.T - current_output  # [out_dim, num_edits]
    
    # 計算 (λI + K^T·K)^{-1}
    KtK = K.T @ K  # [in_dim, in_dim]
    reg_matrix = lambda_reg * torch.eye(in_dim, device=W.device)
    inv_term = torch.linalg.inv(reg_matrix + KtK)  # [in_dim, in_dim]
    
    # 計算更新
    delta = residual @ K @ inv_term  # [out_dim, in_dim]
    
    return delta


# 示範批量編輯
print("批量編輯示範：")
print("="*50)

# 創建權重矩陣
in_dim, out_dim = 10, 8
W = torch.randn(out_dim, in_dim)

# 3 個編輯請求
K = torch.randn(3, in_dim)  # 3 個 keys
V = torch.randn(3, out_dim)  # 3 個 target values

print(f"權重矩陣: {W.shape}")
print(f"Keys: {K.shape}")
print(f"Target Values: {V.shape}")

# 計算更新
delta = batch_rank_update(W, K, V)
W_new = W + delta

# 驗證
print(f"\n驗證編輯效果：")
for i in range(3):
    original = W @ K[i]
    edited = W_new @ K[i]
    target = V[i]
    error = torch.norm(edited - target).item()
    print(f"Edit {i+1}: Error = {error:.6f}")

## 6. 編輯效果評估

### 6.1 評估指標

In [ ]:
class EditEvaluator:
    """
    知識編輯效果評估器
    """
    def __init__(self, model, editor, original_facts: Dict):
        self.model = model
        self.editor = editor
        self.original_facts = original_facts
    
    def evaluate_edit(
        self, 
        subject_id: int, 
        new_object_id: int,
        method: str = "rank_one"
    ) -> Dict:
        """
        評估單次編輯的效果
        
        Returns:
            - efficacy: 編輯是否成功
            - specificity: 對其他知識的影響
        """
        # 恢復原始模型
        self.editor.restore()
        
        # 執行編輯
        self.editor.edit(subject_id, new_object_id, method=method)
        
        results = {
            "method": method,
            "subject": subject_id,
            "old_object": self.original_facts[subject_id],
            "new_object": new_object_id,
        }
        
        # 1. Efficacy: 編輯的 subject 是否正確預測新 object
        pred = self.model.predict(torch.tensor([subject_id]).to(device)).item()
        results["efficacy"] = pred == new_object_id
        results["predicted"] = pred
        
        # 2. Specificity: 其他 subject 是否保持正確
        other_correct = 0
        other_total = 0
        other_results = []
        
        for s, expected_o in self.original_facts.items():
            if s != subject_id:
                pred = self.model.predict(torch.tensor([s]).to(device)).item()
                is_correct = pred == expected_o
                other_correct += int(is_correct)
                other_total += 1
                other_results.append({
                    "subject": s,
                    "expected": expected_o,
                    "predicted": pred,
                    "correct": is_correct
                })
        
        results["specificity"] = other_correct / other_total if other_total > 0 else 1.0
        results["other_subjects"] = other_results
        
        return results


# 重新訓練模型
print("重新訓練模型...")
model = SimpleKnowledgeModel(vocab_size=vocab_size).to(device)
train_knowledge_model(model, knowledge_facts, epochs=500)

# 評估不同編輯方法
editor = KnowledgeEditor(model)
evaluator = EditEvaluator(model, editor, knowledge_facts)

print("\n" + "="*60)
print("編輯方法比較")
print("="*60)

for method in ["fine_tune", "rank_one"]:
    print(f"\n--- {method.upper()} ---")
    results = evaluator.evaluate_edit(
        subject_id=0, 
        new_object_id=60, 
        method=method
    )
    
    print(f"Efficacy: {'✓' if results['efficacy'] else '✗'} "
          f"(Predicted: {results['predicted']}, Target: {results['new_object']})")
    print(f"Specificity: {results['specificity']:.2%}")
    
    print("Other subjects:")
    for other in results['other_subjects']:
        status = '✓' if other['correct'] else '✗'
        print(f"  Subject {other['subject']}: {other['predicted']} "
              f"(expected {other['expected']}) {status}")

In [ ]:
# 視覺化編輯效果
def visualize_edit_comparison():
    """比較不同編輯方法的效果"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    methods = ['Fine-tune', 'Rank-One (ROME-like)']
    efficacy_scores = [0.9, 0.95]  # 模擬數據
    specificity_scores = [0.6, 0.85]
    
    # 1. Efficacy vs Specificity
    x = np.arange(len(methods))
    width = 0.35
    
    bars1 = axes[0].bar(x - width/2, efficacy_scores, width, label='Efficacy', color='steelblue')
    bars2 = axes[0].bar(x + width/2, specificity_scores, width, label='Specificity', color='coral')
    
    axes[0].set_ylabel('Score')
    axes[0].set_title('Edit Quality Comparison')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(methods)
    axes[0].legend()
    axes[0].set_ylim([0, 1.1])
    axes[0].grid(axis='y', alpha=0.3)
    
    # 加上數值標籤
    for bar, val in zip(bars1, efficacy_scores):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                     f'{val:.0%}', ha='center', va='bottom')
    for bar, val in zip(bars2, specificity_scores):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                     f'{val:.0%}', ha='center', va='bottom')
    
    # 2. Number of edits vs Performance
    num_edits = [1, 5, 10, 20, 50, 100]
    finetune_perf = [0.9, 0.7, 0.5, 0.3, 0.15, 0.1]  # Fine-tune 隨編輯增加而下降
    rome_perf = [0.95, 0.92, 0.88, 0.82, 0.75, 0.68]  # ROME 更穩定
    memit_perf = [0.95, 0.93, 0.91, 0.88, 0.85, 0.82]  # MEMIT 最穩定
    
    axes[1].plot(num_edits, finetune_perf, 'o-', label='Fine-tune', linewidth=2)
    axes[1].plot(num_edits, rome_perf, 's-', label='ROME', linewidth=2)
    axes[1].plot(num_edits, memit_perf, '^-', label='MEMIT', linewidth=2)
    
    axes[1].set_xlabel('Number of Edits')
    axes[1].set_ylabel('Overall Performance')
    axes[1].set_title('Scalability of Edit Methods')
    axes[1].legend()
    axes[1].set_xscale('log')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n關鍵觀察：")
    print("1. Fine-tuning 的 efficacy 高但 specificity 低（災難性遺忘）")
    print("2. ROME 在單次編輯時表現優異")
    print("3. MEMIT 在大量編輯時仍能保持較好效果")

visualize_edit_comparison()

## 7. 使用 EasyEdit 進行實作

### 7.1 EasyEdit 簡介

EasyEdit 是一個統一的知識編輯框架，支援多種編輯方法。

```python
# EasyEdit 使用範例（概念展示）
# 安裝：pip install easyeditor

from easyeditor import BaseEditor, ROMEHyperParams

# 載入預訓練模型
hparams = ROMEHyperParams.from_hparams('hparams/ROME/gpt2-xl.yaml')
editor = BaseEditor.from_hparams(hparams)

# 定義編輯
prompts = ['The Prime Minister of UK is']
ground_truth = ['Keir Starmer']
target_new = ['Keir Starmer']
subject = ['UK']

# 執行編輯
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=True
)

print(metrics)  # {'efficacy': 1.0, 'generalization': 0.95, ...}
```

In [ ]:
# 模擬 EasyEdit 風格的 API
class SimpleEditor:
    """
    簡化版知識編輯器（模擬 EasyEdit API）
    """
    def __init__(self, model, method: str = "ROME"):
        self.model = model
        self.method = method
        self.original_state = deepcopy(model.state_dict())
        
    def edit(
        self,
        prompts: List[str],
        target_new: List[str],
        subject: List[str],
        keep_original_weight: bool = True
    ) -> Dict:
        """
        執行知識編輯
        
        Args:
            prompts: 輸入 prompts
            target_new: 新的目標答案
            subject: 主詞
            keep_original_weight: 是否保留原始權重（用於比較）
        
        Returns:
            metrics: 編輯效果指標
        """
        print(f"\nEditing with method: {self.method}")
        print(f"Number of edits: {len(prompts)}")
        
        for i, (prompt, target, subj) in enumerate(zip(prompts, target_new, subject)):
            print(f"  Edit {i+1}: '{prompt}' -> '{target}' (subject: {subj})")
        
        # 模擬編輯過程
        metrics = {
            "efficacy": 0.95,
            "generalization": 0.88,
            "specificity": 0.92,
            "fluency": 0.97,
            "method": self.method
        }
        
        return metrics
    
    def restore(self):
        """恢復原始權重"""
        self.model.load_state_dict(deepcopy(self.original_state))


# 使用範例
print("模擬 EasyEdit 使用流程：")
print("="*60)

editor = SimpleEditor(model, method="ROME")

metrics = editor.edit(
    prompts=["The Prime Minister of UK is"],
    target_new=["Keir Starmer"],
    subject=["UK Prime Minister"]
)

print(f"\nEdit Metrics:")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2%}")
    else:
        print(f"  {key}: {value}")

## 8. 練習題

### 練習 1：實作 Causal Tracing

In [ ]:
# 練習 1：實作簡化版 Causal Tracing
def causal_trace(
    model: nn.Module,
    subject_id: int,
    clean_answer: int,
    corrupted_subject_id: int
) -> Dict[str, float]:
    """
    TODO: 實作 Causal Tracing 來找出哪一層對知識最重要
    
    步驟：
    1. 用 clean input 得到正確答案的機率
    2. 用 corrupted input 得到機率（應該下降）
    3. 逐層恢復 clean 的 activation，看哪層恢復效果最大
    
    Args:
        model: 要分析的模型
        subject_id: 原始 subject
        clean_answer: 正確答案
        corrupted_subject_id: 被干擾的 subject
    
    Returns:
        每一層的「重要性分數」
    """
    # 提示：
    # 1. 使用 forward hook 來擷取中間 activation
    # 2. 計算恢復特定層後的輸出變化
    pass

print("練習 1：實作 causal_trace 函數")
print("這個函數可以幫助我們找出模型中儲存特定知識的關鍵層")

### 練習 2：多層編輯

In [ ]:
# 練習 2：實作將編輯分散到多層的方法
class MultiLayerEditor:
    """
    TODO: 實作將編輯分散到多層的編輯器（類似 MEMIT）
    
    想法：
    - 不要把整個編輯放在單一層
    - 將 target value 分解，分散到多層的小更新
    - 這樣可以減少對單一層的干擾
    """
    def __init__(self, model: nn.Module, layers_to_edit: List[int]):
        self.model = model
        self.layers_to_edit = layers_to_edit
    
    def edit(self, subject_id: int, target_value: torch.Tensor) -> Dict:
        """
        TODO: 將編輯分散到多層
        
        提示：
        1. 將 target_value 分解為 len(layers_to_edit) 個部分
        2. 對每一層應用較小的更新
        3. 返回編輯的統計資訊
        """
        pass

print("練習 2：實作 MultiLayerEditor")
print("這種方法可以提高大量編輯時的穩定性")

### 練習 3：編輯效果可視化

In [ ]:
# 練習 3：分析編輯對模型權重的影響
def analyze_weight_change(
    original_weights: Dict[str, torch.Tensor],
    edited_weights: Dict[str, torch.Tensor]
) -> Dict:
    """
    TODO: 分析編輯前後權重的變化
    
    分析項目：
    1. 每一層權重的 L2 變化量
    2. 變化最大的層
    3. 權重變化的分布（直方圖）
    
    這有助於理解編輯的「代價」和「範圍」
    """
    # 提示：
    # 1. 遍歷所有層的權重
    # 2. 計算差異的各種統計量
    # 3. 繪製視覺化圖表
    pass

print("練習 3：實作 analyze_weight_change 函數")
print("這可以幫助我們理解不同編輯方法的影響範圍")

## 9. 總結

### 9.1 模型編輯方法總覽

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     知識編輯方法比較                                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  方法          │ 優點                    │ 缺點                         │
│  ───────────────────────────────────────────────────────────────────── │
│  Fine-tuning  │ 簡單直接                │ 災難性遺忘                    │
│               │ 不需要特殊實作          │ 需要大量資料                  │
│  ───────────────────────────────────────────────────────────────────── │
│  ROME         │ 精準編輯單一知識         │ 一次只能編輯一條              │
│               │ 保持其他知識             │ 需要找到關鍵層                │
│  ───────────────────────────────────────────────────────────────────── │
│  MEMIT        │ 可批量編輯               │ 計算較複雜                    │
│               │ 擴展性好                 │ 需要更多記憶體                │
│  ───────────────────────────────────────────────────────────────────── │
│  MEND         │ 學習如何編輯             │ 需要訓練 hypernetwork         │
│               │ 快速適應                 │ 泛化能力有限                  │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 9.2 實際應用建議

1. **小規模編輯**（< 10 條）：ROME
2. **中等規模編輯**（10-100 條）：MEMIT
3. **頻繁更新場景**：考慮 RAG 結合編輯
4. **安全性要求高**：編輯後要仔細評估副作用

In [ ]:
print("="*60)
print("模型知識編輯 - 學習完成！")
print("="*60)
print("\n你已經學會：")
print("✓ 理解知識在 Transformer 中的儲存方式")
print("✓ ROME 的核心原理（Rank-One Update）")
print("✓ MEMIT 的批量編輯方法")
print("✓ 編輯效果的評估指標")
print("✓ 實作簡單的知識編輯器")
print("\n下一步學習建議：")
print("1. 使用 EasyEdit 在真實 LLM 上實驗")
print("2. 研究 Causal Tracing 的細節")
print("3. 探索編輯的可逆性和持久性")
print("4. 了解編輯與對齊的關係")